In [1]:
import numpy as np 

A = np.array([[1, 2, 3], [4, 5, 6]], dtype=float)
B = np.array([[7, 8], [9, 10], [11, 12]], dtype=float)
A,B

(array([[1., 2., 3.],
        [4., 5., 6.]]),
 array([[ 7.,  8.],
        [ 9., 10.],
        [11., 12.]]))

In [2]:
import taichi as ti
import numpy as np

# Initialize Taichi
ti.init()

# Create a Taichi ndarray
taichi_array = ti.ndarray(ti.f32, shape=(10, 10))

# Perform operations on the Taichi ndarray (if necessary)
taichi_array[3, 4] = 10

# Convert to a NumPy array
numpy_array = taichi_array.to_numpy()
print(numpy_array)


[Taichi] version 1.7.0, llvm 15.0.4, commit 2fd24490, linux, python 3.10.12


[I 12/29/23 10:59:47.585 7900] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


[Taichi] Starting on arch=x64
[[ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0. 10.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]]


In [3]:
import taichi as ti
import numpy as np
ti.init()

a = np.zeros((5, 5))

@ti.kernel
def test(a: ti.types.ndarray()):
    for i in range(a.shape[0]):  # a parallel for loop
        for j in range(a.shape[1]):
            a[i, j] = i + j

test(a)
print(a)

[Taichi] Starting on arch=x64
[[0. 1. 2. 3. 4.]
 [1. 2. 3. 4. 5.]
 [2. 3. 4. 5. 6.]
 [3. 4. 5. 6. 7.]
 [4. 5. 6. 7. 8.]]


In [32]:
import taichi as ti
import numpy as np

ti.init(arch=ti.cpu)  # Use CPU by default

def matrix_multiply_numpy_to_taichi(A: np.ndarray, B: np.ndarray) -> np.ndarray:
    C = ti.field(shape=(A.shape[0], B.shape[1]), dtype=ti.f64)

    # Convert the numpy arrays to Taichi ndarrays
    A_ti = ti.field(shape=A.shape, dtype=ti.f64)
    B_ti = ti.field(shape=B.shape, dtype=ti.f64)
    A_ti.from_numpy(A)
    B_ti.from_numpy(B)
    sum = ti.field(dtype=ti.f64, shape=())

    @ti.kernel
    def _taichi_compute():
        for i in range(A.shape[0]):
            for j in range(B.shape[1]):
                sum[None] = 0.0
                for k in range(A.shape[1]):
                    sum[None] += A_ti[i, k] * B_ti[k, j]
                C[i,j] = sum[None]

    _taichi_compute()
    return C.to_numpy()



[Taichi] Starting on arch=x64


In [33]:
A, B

(array([[1., 2., 3.],
        [4., 5., 6.]]),
 array([[ 7.,  8.],
        [ 9., 10.],
        [11., 12.]]))

In [34]:
%%time
# Example usage
C = matrix_multiply_numpy_to_taichi(A, B)
C


CPU times: user 117 ms, sys: 5.4 ms, total: 122 ms
Wall time: 120 ms


array([[ 58.,  64.],
       [139., 154.]])

In [38]:
A1 = np.random.randn(20, 30)
B1 = np.random.randn(30, 40)
A1.shape, B1.shape

((20, 30), (30, 40))

In [39]:
%%time
# Example usage
C1 = matrix_multiply_numpy_to_taichi(A1, B1)
C1.shape


CPU times: user 282 ms, sys: 5.9 ms, total: 288 ms
Wall time: 286 ms


(20, 40)